In [ ]:
from __future__ import print_function, division
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import math
import numpy as np
import wx
import os
import cv2
import pywt
from pylab import imread
from PIL import Image, ImageDraw
import pywt
from pywt import dwt2, idwt2

In [ ]:
# Load Images
def get_path(wildcard):
    app = wx.App(None)
    style = (wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) | wx.FD_MULTIPLE
    dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        path = dialog.GetPaths()
    else:
        path = None
    dialog.Destroy()
    return path

# Init lists of Image, Name
listImage = []
listName = []
i = 0
for link in get_path('*.*'):
    name = link
    name = os.path.basename(link)
    if (os.path.splitext(name)[1]=='.jpg') or (os.path.splitext(name)[1]=='.png'):
        name = os.path.splitext(name)[0]
        image = imread(link)
        listName.append(name)
        listImage.append(image)
    
def psnr(img1, img2):
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64) 
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

# Compress
def Compress(image,threshold):
    thresh = threshold
    # Save Original Size
    if len(image.shape)>2:
        img = Image.fromarray(image.astype('uint8'), 'RGB')
        ImageDraw.Draw(img)
        img.save('out.png')
        s1 = os.stat('out.png').st_size 
    else:
        plt.imsave('out.png', image, cmap='gray')
        s1 = os.stat('out.png').st_size
    
    if len(image.shape)>2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        H_Image = image[:,:,0];
        S_Image = image[:,:,1];
        V_Image = image[:,:,2];
        
        c = pywt.wavedec2(H_Image, 'bior4.4',level = 3)
        arr, slices = pywt.coeffs_to_array(c)
        dwt_thresh = arr*(abs(arr)>(thresh*np.max(arr)))
        c = pywt.array_to_coeffs(dwt_thresh, slices, output_format='wavedecn')
        A = pywt.waverecn(c,'bior4.4')
        
        c = pywt.wavedec2(S_Image, 'bior4.4',level = 3)
        arr, slices = pywt.coeffs_to_array(c)
        dwt_thresh = arr*(abs(arr)>(thresh*np.max(arr)))
        c = pywt.array_to_coeffs(dwt_thresh, slices, output_format='wavedecn')
        B = pywt.waverecn(c,'bior4.4')
        
        c = pywt.wavedec2(V_Image, 'bior4.4',level = 3)
        arr, slices = pywt.coeffs_to_array(c)
        dwt_thresh = arr*(abs(arr)>(thresh*np.max(arr)))
        c = pywt.array_to_coeffs(dwt_thresh, slices, output_format='wavedecn')
        C2 = pywt.waverecn(c,'bior4.4')
        
        New_Image = cv2.merge((A,B,C2)).astype('uint8')
        New_Image = cv2.cvtColor(New_Image, cv2.COLOR_HSV2BGR)
        plt.imsave('out2.png', New_Image)
        s2 = os.stat('out2.png').st_size
        plt.imshow(New_Image)
        print('Compress Quality: {} dB'.format(psnr(image,New_Image)))
    else:
        c = pywt.wavedec2(image, 'bior4.4',level = 3)
        arr, slices = pywt.coeffs_to_array(c)
        dwt_thresh = arr*(abs(arr)>(thresh*np.max(arr)))
        c = pywt.array_to_coeffs(dwt_thresh, slices, output_format='wavedecn')
        New_Image = pywt.waverecn(c,'bior4.4')
        plt.imsave('out2.png', New_Image, cmap='gray')
        s2 = os.stat('out2.png').st_size
        plt.imshow(New_Image,cmap='gray')
        print('Compress Quality: {} dB'.format(psnr(image,New_Image)))
    
    print('Compress Ratio: {}'.format(s1/s2))
# Choose Image
def ChosenImage(image_name):
    for i in range(0,len(listImage)):
        if listName[i]==image_name:
            image_number = i
            image = listImage[i]
            width = image.shape[1]
    interact(Compress, image=fixed(image), threshold = widgets.FloatSlider(min=0, max = 1, step=0.001, value=0.01))
interact(ChosenImage,image_name=listName)